# Decision Trees Exercise
In this exercise you will show that ID3 is sub-optimal. Implement a simple version of Decision Tree, and will then apply a Decision Tree classsifier on the MNIST hand written digits dataset that we already saw.


In [15]:
(2/3)*np.log2(2/3)

-0.38997500048077083

In [16]:
2*(.5)*np.log2(.5)

-1.0

## Suboptimality of ID3
Consider the following training set, where $\mathcal{X} = \{0, 1\}^3$ and $\mathcal{Y} =\{0, 1\}$:

$$
\begin{aligned}
((1, 1, 1), 1)\\
((1, 0, 0), 1)\\
((1, 1, 0), 0)\\
((0, 0, 1), 0)
\end{aligned}
$$

Suppose we wish to use this training set in order to build a decision tree of depth 2 (i.e. for each
input we are allowed to ask two questions of the form "$x_i = 0$?" before deciding on the label).

1. Suppose we run the ID3 algorithm up to depth 2 (namely, we pick the root node and its
children according to the algorithm, but instead of keeping on with the recursion, we stop
and pick leaves according to the majority label in each subtree, once we reach depth 2). 
Assume that the subroutine used to measure the quality of each feature is based on the information gain, and that if two features get the same score, one of them is picked arbitrarily. 
Show that the training error of the resulting decision tree is at least 1/4.
2. Find a decision tree of depth 2, which attains zero training error.


The decision tree based on ID3 will give us a split first on $X_1$ (the other two features will give no information gain), followed by either of the two other features, which both give the same information gain.  Assuming the second split is on $X_2$, we can see that if the answer is $0$ we will get the correct $Y$ of 1 but if the answer is $1$ we are left with a leaf with both

#### Answer
Put your answer here...

## Implementing Decision Tree From Scratch
In this exercise you will need to implement a simple version of Decision Tree from scratch. Your decision tree will handle **continuous input and output** (this should actually work also for binary input attributes).

* Compelete the skeleton class below
  - `X` is a matrix of data values (rows are samples, columns are attributes)
  - `y` is a vector of corresponding target values
  - `min_leaf` is the minimal number of samples in each leaf node
  
* For splitting criterion, use either **"Train Squared Error Minimization (Reduction in Variance)"** or **"Train Absolute Error Minimization"** (choose one). Whatever you choose, make sure you implement the splitting point decision efficiently (in $O(nlgn)$ time).

* The `predict` function will use mean of the target values in the leaf node matching each row of the given `X`. The result is a vector of predictions matching the number of rows in `X`.

* To check your decision tree implementation, use the boston dataset (`from sklearn.datasets import load_boston`) split the data set into train and test using (`from sklearn.model_selection import train_test_split`)

  - Use the following to estimate what are the best hyper parameters to use for your model
```
    for min_leaf in [1,5,10,100]:
      dt = DecisionTree(X, y, n, sz, min_leaf)
      mse = # mean square error over test set
      print("min_leaf:{0} --- oob mse: {1}".format(min_leaf, mse))
```
  
  - Using your chosen hyperparameters as a final model, plot the predictions vs. true values of all the samples in the training set . Use something like:
  ```
  y_hat = dt.predict(X_train)  # forest is the chosen model
  plt.scatter(y_hat, y_test)
  ```

In [3]:
import numpy as np
import pandas as pd

In [4]:
from sklearn.datasets import load_boston
X,y = load_boston(return_X_y=True)

In [5]:
def sse(samples):
    return np.var(samples)*len(samples)

In [8]:
n_trees=5
sample_sz=20
min_lf = 10


In [78]:
# list_of_cutoffs=list()
# list_of_coords=list()
# coords=[]
# def make_a_split(sampleX,sampleY,sample_sz,min_leaf):
#     cutoffs=list()
#     for feature in sampleX.T:   
#         unique_cutoffs=np.unique(feature)[:-1]
#         cutoffs.append(np.random.choice(unique_cutoffs,replace=False,size=min(sample_sz,len(unique_cutoffs))))

#     variances = np.ones((sampleX.shape[1],sample_sz))*sse(sampleY)
#     for feature,cutoff_array in enumerate(cutoffs):
#         for sample,cutoff in enumerate(cutoff_array):
#             variances[feature,sample]=np.nansum([sse(sampleY[sampleX[:,feature]<=cutoff]),sse(sampleY[sampleX[:,feature]>cutoff])])
#     feature,cutoff_num = np.unravel_index(variances.argmin(), variances.shape)
    
#     cutoff_point = cutoffs[feature][cutoff_num]
#     list_of_cutoffs.append([feature,cutoff_point])
#     print(coords)
#     list_of_coords.append(coords)
#     index1=sampleX[:,feature]>cutoff_point
#     index2=sampleX[:,feature]<=cutoff_point
    
#     X1,X2,y1,y2 = sampleX[index1,:],sampleX[index2,:],sampleY[index1],sampleY[index2]
#     if len(y1)>min_leaf:
#         X_n,y_n,X_n2,y_n2,coords=make_a_split(X1,y1,sample_sz,min_leaf,coords+['a'])
#         print('going right')
#     if len(y2)>min_leaf:
#         X_n,y_n,X_n2,y_n2,coords=make_a_split(X2,y2,sample_sz,min_leaf,coords+['b'])
#         print('going left')
#     return X1,y1,X2,y2,coords

In [307]:
def make_a_split(sampleX,sampleY,sample_sz):
    cutoffs=list()
    for feature in sampleX.T:   
        unique_cutoffs=np.unique(feature)[:-1]
        cutoffs.append(np.random.choice(unique_cutoffs,replace=False,size=min(sample_sz,len(unique_cutoffs))))
    variances = np.ones((sampleX.shape[1],sample_sz))*sse(sampleY)
    for feature,cutoff_array in enumerate(cutoffs):
        for sample,cutoff in enumerate(cutoff_array):
            variances[feature,sample]=np.nansum([sse(sampleY[sampleX[:,feature]<=cutoff]),sse(sampleY[sampleX[:,feature]>cutoff])])
    feature,cutoff_num = np.unravel_index(variances.argmin(), variances.shape)
    cutoff_point = cutoffs[feature][cutoff_num]
    f_c = [feature,cutoff_point]
    index1=sampleX[:,feature]>cutoff_point
    index2=sampleX[:,feature]<=cutoff_point
    X1,X2,y1,y2 = sampleX[index1,:],sampleX[index2,:],sampleY[index1],sampleY[index2]
    return X1,y1,X2,y2,f_c
def split_recursive(X,y,sample_sz,min_leaf,max_tree,coords,list_of_cutoffs):
    X1,y1,X2,y2,f_c=make_a_split(X,y,sample_sz)
    if len(y1)>min_leaf and len(list_of_cutoffs)<(max_tree-1):
        split_recursive(X1,y1,sample_sz,min_leaf,max_tree,coords=coords+['a'],list_of_cutoffs=list_of_cutoffs+[f_c])
    else:
        lol.append([list_of_cutoffs+[f_c],coords+['a'],y1.mean()])
    if len(y2)>min_leaf and len(list_of_cutoffs)<(max_tree-1):
        split_recursive(X2,y2,sample_sz,min_leaf,max_tree,coords=coords+['b'],list_of_cutoffs=list_of_cutoffs+[f_c])
    else:
        lol.append([list_of_cutoffs+[f_c],coords+['b'],y2.mean()])

list_of_cutoffs=list()
list_of_coords=list()
lol=[]
splits=[]
split_recursive(X,y,20,100,max_tree=3,coords=[],list_of_cutoffs=[])

In [308]:
def predict(x2check,lol):
    for i in lol:
        stop=False
        for n,pair in enumerate(i[0]):
            if stop==False:
                if (x2check[pair[0]]>pair[1] and i[1][n]=="a") or (x2check[pair[0]]<=pair[1] and i[1][n]=="b") :
                    if n==len(i[0])-1:
                        return (i[2])
                else:
                    stop = True
                
predict(X[87],lol)

22.854509803921566

In [333]:
class DecisionTree():
    def __init__(self, X, y, n_trees, sample_sz, min_leaf):
        self.X,self.y,self.n_trees,self.sample_sz,self.min_leaf=X,y, n_trees, sample_sz, min_leaf
        self.lol=[]
    @staticmethod
    def sse(samples):
        return np.var(samples)*len(samples)
    @staticmethod
    def make_a_split(sampleX,sampleY,min_leaf):
        print(sampleX)
        cutoffs=list()
        for feature in sampleX.T:   
            unique_cutoffs=np.unique(feature)[:-1]
            cutoffs.append(np.random.choice(unique_cutoffs,replace=False,size=min(self.sample_sz,len(unique_cutoffs))))
        variances = np.ones((sampleX.shape[1],sample_sz))*sse(sampleY)
        for feature,cutoff_array in enumerate(cutoffs):
            for sample,cutoff in enumerate(cutoff_array):
                variances[feature,sample]=np.nansum([sse(sampleY[sampleX[:,feature]<=cutoff]),sse(sampleY[sampleX[:,feature]>cutoff])])
        feature,cutoff_num = np.unravel_index(variances.argmin(), variances.shape)
        cutoff_point = cutoffs[feature][cutoff_num]
        f_c = [feature,cutoff_point]
        index1=sampleX[:,feature]>cutoff_point
        index2=sampleX[:,feature]<=cutoff_point
        X1,X2,y1,y2 = sampleX[index1,:],sampleX[index2,:],sampleY[index1],sampleY[index2]
        return X1,y1,X2,y2,f_c
    def split_recursive(X,y,sample_sz,min_leaf,n_trees,coords=[],list_of_cutoffs=[]):
        X1,y1,X2,y2,f_c=make_a_split(X,y,sample_sz)
        if len(y1)>min_leaf and len(list_of_cutoffs)<(n_trees-1):
            split_recursive(X1,y1,sample_sz,min_leaf,n_trees,coords=coords+['a'],list_of_cutoffs=list_of_cutoffs+[f_c])
        else:
            self.lol.append([list_of_cutoffs+[f_c],coords+['a'],y1.mean()])
        if len(y2)>min_leaf and len(list_of_cutoffs)<(n_trees-1):
            split_recursive(X2,y2,sample_sz,min_leaf,n_trees,coords=coords+['b'],list_of_cutoffs=list_of_cutoffs+[f_c])
        else:
            self.lol.append([list_of_cutoffs+[f_c],coords+['b'],y2.mean()])
    def fit(self):
        self.split_recursive(self.X,self.y,self.sample_sz,self.min_leaf,self.n_trees)
    def predict(self,x2check):
        for i in self.lol:
            stop=False
            for n,pair in enumerate(i[0]):
                if stop==False:
                    if (x2check[pair[0]]>pair[1] and i[1][n]=="a") or (x2check[pair[0]]<=pair[1] and i[1][n]=="b") :
                        if n==len(i[0])-1:
                            return (i[2])
                    else:
                        stop = True



In [334]:
dt = DecisionTree(X,y,20,100,3)
dt.fit()

AttributeError: 'DecisionTree' object has no attribute 'T'

In [302]:
dt.lol

[]

In [299]:
dt.predict(X[10])

## Using Decision Treefor Digits Classification
Remeber the MNIST dataset used - you will now test the power of decision trees on this problem.
This time you are given a free hand in choosing the test and train set sizes, model parameters (such as gain function and constraints over the trees) and features (whether to use binary pixel values or the original continous gray value).
- Choose which model parameters you wish to optimize, explain how would you do that, and find a model which you believe would have the minimal generalization error --- do this for both a single decision tree model, and a random forest.
  - You can use `sklearn.tree.DecisionTreeClassifier`
- Once you are satisfied with the model parameters, plot the importance of each of the pixels to the final decision.
- Last, estimate the class assignment probabilities for all the correctly classified and misclassified examples in your test data.
- Discuss your results.

In [ ]:
# code and answer go here